In [52]:
import tensorflow as tf

def setup_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        try:
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
            print("config augmentation allocation mémoire gpu activée")
        except RuntimeError as e:
            print(e)

setup_gpu()

config augmentation allocation mémoire gpu activée


In [53]:
import keras
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sea
import sklearn
import scipy as sc
import nltk as nltk
import statsmodels as statsmodels
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [54]:
print("Version de TensorFlow :", tf.__version__)

Version de TensorFlow : 2.16.2


In [55]:
print("Version de Keras :", keras.__version__)

Version de Keras : 3.4.1


## Panneau de configuration

In [56]:
project_path = "/Users/carolus/Documents/WorkSpace/green_ia/cch/"
file_nbr = '03' # numéro d'identification des csv à générer 
openfoodfact_csv_version = '00' # version csv open food facts à utiliser

In [57]:
# import du csv de données openfoodfacts
openfoodfacts_csv = project_path + f"data/openfoodfacts_{openfoodfact_csv_version}.csv"
df = pd.read_csv(openfoodfacts_csv)

/var/folders/__/7s1jl_m52b5f2glx35x7vpcc0000gn/T/ipykernel_40293/4033427239.py:3: DtypeWarning: Columns (0,11,17,23,24,25,31,32,33,34,35,45,46,50,52,53,56,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(openfoodfacts_csv)


# Analyse des données 

In [58]:
# affiche le nom de toutes les colonnes du df 
column_list = list(df.columns)
print(f"nbr col: {len(column_list)}, liste: {column_list}")

nbr col: 206, liste: ['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'last_modified_by', 'last_updated_t', 'last_updated_datetime', 'product_name', 'abbreviated_product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'packaging_en', 'packaging_text', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'origins_en', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'ingredients_tags', 'ingredients_analysis_tags', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutrition_data', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'nutriscore_score', 'nutriscore_grade', 'nova_group'

In [59]:
# afficher % de nan / colonne:
total_rows = df.shape[0]
nan_counts = df.isna().sum()
nan_percentage = (nan_counts / total_rows) * 100
nan_df = pd.DataFrame({'column_name': nan_percentage.index, 'percentage_nan': nan_percentage.values})

nan_percent_range = [(0, 10), (10, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100)]
grouped = nan_df.groupby(pd.cut(nan_df['percentage_nan'], bins=[tranche[0] for tranche in nan_percent_range + [(100,)]]))

for tranche, group in grouped:
    print(f"range {tranche}:")
    print(group['column_name'].tolist())
    print()

range (0, 10]:
['creator', 'last_modified_by', 'last_updated_t', 'last_updated_datetime', 'product_name', 'countries', 'countries_tags', 'countries_en', 'nutriscore_grade', 'pnns_groups_1', 'pnns_groups_2', 'states', 'states_tags', 'states_en', 'ecoscore_grade', 'completeness']

range (10, 20]:
['last_image_t', 'last_image_datetime']

range (20, 30]:
['image_url', 'image_small_url', 'energy-kcal_100g', 'energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'proteins_100g']

range (30, 40]:
['salt_100g', 'sodium_100g']

range (40, 50]:
['brands', 'brands_tags', 'image_nutrition_url', 'image_nutrition_small_url']

range (50, 60]:
['categories', 'categories_tags', 'categories_en', 'main_category', 'main_category_en']

range (60, 70]:
['quantity', 'ingredients_analysis_tags', 'nutriscore_score', 'food_groups', 'food_groups_tags', 'food_groups_en', 'nutrient_levels_tags', 'unique_scans_n', 'popularity_tags', 'fiber_100g', 'nutrition-score-fr_100g']

range (70,

/var/folders/__/7s1jl_m52b5f2glx35x7vpcc0000gn/T/ipykernel_40293/3943661832.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = nan_df.groupby(pd.cut(nan_df['percentage_nan'], bins=[tranche[0] for tranche in nan_percent_range + [(100,)]]))


# Traitement des données 

In [60]:
# récupérer les colonnes intéressantes 
col_classi = [
    'product_name',
    'pnns_groups_1',
    'pnns_groups_2',
    'energy-kcal_100g',
    'fat_100g',
    'saturated-fat_100g',
    'carbohydrates_100g',
    'sugars_100g',
    'proteins_100g',
    'salt_100g',
    'sodium_100g',
    'brands_tags',
    'image_url'
]
cat_df = df[col_classi] 
cat_df.tail(3)

,product_name,pnns_groups_1,pnns_groups_2,energy-kcal_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g,brands_tags,image_url
3295156,Test Product Product,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mtr,NaN
3295157,Light & Free SKYR A BOIRE,Milk and dairy products,Dairy desserts,24.0,28.0,13.0,70.0,49.0,2.0,3.0,1.2,NaN,https://images.openfoodfacts.org/images/produc...
3295158,Salatgurke,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,edeka,https://images.openfoodfacts.org/images/produc...


In [61]:
# renommer les colonnes 
rename_col = {
    'pnns_groups_1': 'pnns1',
    'energy-kcal_100g': 'kcal', 
    'fat_100g': 'fat',
    'saturated-fat_100g': 'sat_fat',
    'carbohydrates_100g': 'carbohyd',
    'sugars_100g': 'sugar',
    'proteins_100g': 'prot',
    'salt_100g': 'salt',
    'sodium_100g': 'sodium',
    'brands_tags': 'brand',
    'pnns_groups_2': 'pnns2',
    'image_url': 'img_url',
    'product_name': 'name',
}
cat_df = cat_df.rename(columns=rename_col)
cat_df.tail(3)

,name,pnns1,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand,img_url
3295156,Test Product Product,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mtr,NaN
3295157,Light & Free SKYR A BOIRE,Milk and dairy products,Dairy desserts,24.0,28.0,13.0,70.0,49.0,2.0,3.0,1.2,NaN,https://images.openfoodfacts.org/images/produc...
3295158,Salatgurke,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,edeka,https://images.openfoodfacts.org/images/produc...


In [62]:
# suppresion des lignes où pnns2 = unknow ou NaN
cat_df = cat_df.dropna(subset = ["pnns2"])
cat_df = cat_df[cat_df['pnns2'] != 'unknown']

# mélange des lignes aléatoirement 
cat_df = cat_df.sample(frac=1).reset_index(drop=True)

cat_df.tail(3)

,name,pnns1,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand,img_url
1169121,Bonito en aceite,Fish Meat Eggs,Fish and seafood,197.0,1.6,0.50,0.5,0.5,28.0,0.900,0.36,izaro,https://images.openfoodfacts.org/images/produc...
1169122,Tasmanian Apple Cider Pork Sausages,Fish Meat Eggs,Processed meat,267.0,22.4,8.00,4.5,1.7,12.5,1.175,0.47,NaN,https://images.openfoodfacts.org/images/produc...
1169123,Black Beans,Cereals and potatoes,Legumes,80.0,0.4,0.16,16.0,0.0,6.4,0.200,0.08,western-family,https://images.openfoodfacts.org/images/produc...


In [69]:
# afficher les lignes pour les quelles url img et nom produit ne sont pas vides au même moment  
result = cat_df[cat_df['img_url'].isna() & cat_df['name'].notna()]
result.head(5)

,name,pnns1,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand,img_url
1,Orange Drink,Beverages,Sweetened beverages,4.600000e-16,0.000000e+00,0.000000e+00,1.724306e-08,1.893684e-06,9.371250e-12,1.250000e-16,1.250000e-16,prairie-farms-dairy-inc,NaN
4,"Borden, finely shredded parmesan cheese",Milk and dairy products,Cheese,3.930000e-15,4.201792e-09,2.447949e-08,5.274869e-09,1.545865e-07,4.552875e-11,4.465625e-14,4.465625e-14,borden,NaN
8,Apple Pie Dessert Bites,Sugary snacks,Sweets,4.150000e-15,1.614829e-09,1.486744e-08,1.153231e-07,9.956915e-06,9.371250e-12,2.656250e-15,2.656250e-15,good-delish,NaN
17,Blackcurrant Fruit Spread,Sugary snacks,Sweets,2.110000e-15,0.000000e+00,NaN,7.776369e-08,7.477348e-06,9.371250e-12,0.000000e+00,0.000000e+00,st-dalfour,NaN
32,"Marinela, barritas, moras raspberry/blackberry...",Sugary snacks,Biscuits and cakes,4.290000e-15,2.626672e-09,1.631458e-08,9.499198e-08,4.571122e-06,1.338750e-11,4.468750e-15,4.468750e-15,"marinela,bimbo-bakeries-usa-inc",NaN


In [63]:
# normalisation des données numériques 
# supprimer toutes les valeurs < 0 et > 100
col_to_norm = ['kcal', 'fat', 'sat_fat', 'carbohyd', 'sugar', 'prot', 'salt', 'sodium']
scaler = MinMaxScaler()
cat_df[col_to_norm] = scaler.fit_transform(cat_df[col_to_norm])
cat_df.tail(3)

,name,pnns1,pnns2,kcal,fat,sat_fat,carbohyd,sugar,prot,salt,sodium,brand,img_url
1169121,Bonito en aceite,Fish Meat Eggs,Fish and seafood,1.970000e-15,2.353121e-10,7.616517e-10,7.387772e-10,2.318797e-07,4.087125e-11,1.125000e-14,1.125000e-14,izaro,https://images.openfoodfacts.org/images/produc...
1169122,Tasmanian Apple Cider Pork Sausages,Fish Meat Eggs,Processed meat,2.670000e-15,3.294370e-09,1.218643e-08,6.648995e-09,4.173835e-07,2.343375e-11,1.468750e-14,1.468750e-14,NaN,https://images.openfoodfacts.org/images/produc...
1169123,Black Beans,Cereals and potatoes,Legumes,8.000000e-16,5.882803e-11,2.437286e-10,2.364087e-08,1.545865e-07,1.657125e-11,2.500000e-15,2.500000e-15,western-family,https://images.openfoodfacts.org/images/produc...


# Génération df predict catégories

In [64]:
# split des df de train, test et valid
cat_train_df, cat_test_df = train_test_split(cat_df, test_size=0.1, random_state=42)
cat_train_df, cat_valid_df = train_test_split(cat_train_df, test_size=0.05, random_state=42) 

# sauvegarde au format csv 
cat_train_df.to_csv(project_path + f'data/cat_train_df_{file_nbr}.csv', index=False, sep=";")
cat_test_df.to_csv(project_path + f'data/cat_test_df_{file_nbr}.csv', index=False, sep=";")
cat_valid_df.to_csv(project_path + f'data/cat_valid_df_{file_nbr}.csv', index=False, sep=";")